<a href="https://colab.research.google.com/github/salujarohit/DSB-1/blob/master/twitter_tweepy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tweepy

In [ ]:
import tweepy
import pandas as pd
import numpy as np
import math
import re
import pandas as pd

In [ ]:
#Enter your keys
consumer_key = "1keWXFLMoUmnYoLYW1h9Rc2KF"
consumer_secret = "4xvEQPOC7R7fABXrDeqSmKWiiQIO38RiorkBlU8NDCrDMS1mDV"
access_token = "3734940073-DqphcSYiyDL4i4PSvLErV1G3ca6ETC5BpahGDXi"
access_token_secret = "0N5Q8Q0uBWLqiPiSC05iR6qTfxVX0AiErh80B7tZyYJ21"
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [ ]:
# tweets_count = api.get_user(id=username)._json['statuses_count']
# tweets_per_page = 200
# tweets_pages = math.ceil(tweets_count / tweets_per_page);
# tweets_all = []
# for page in range(1,tweets_pages+1):
#   tweets = api.user_timeline(id=username, count=200, page=page)
#   print('page:' + str(page) + " len:" + str(len(tweets)))
#   for tweet in tweets:
#     tweets_all.append(tweet._json['created_at'])
#   print('total: ', str(len(tweets_all)))
# print(len(tweets_all))

In [ ]:
#tweet._json

In [ ]:
# tweets = api.user_timeline(id=username,count = 200, page=3)
# tweets[0]._json['text']

In [ ]:
# #user object 
# user = api.get_user(id=username)
# status_count = user._json['statuses_count']
# print(status_count)

# Pull Tweets Function

In [ ]:
def pulltweets(username):
  """Pass twitter username in quotes
  
  Returns: A cleaned text file of the tweets
  """
  count = -1
  try:    
  # Creation of query method using parameters
    tweets = tweepy.Cursor(api.user_timeline,id=username).items(count)
  except BaseException as e:
    print('failed on_status,',str(e))
    time.sleep(3)
  
  tweets_list = [[tweet.created_at, tweet.id, tweet.text] for tweet in tweets] # Pulling information from tweets iterable object
   
  tweets_df = pd.DataFrame(tweets_list, columns=['Date/Time','ID',"Tweets"]) #tweets dataframe
  print('Total number of tweets pulled ' + str(len(tweets_list)))
  print('Shape of tweeets dataframe ' + str(tweets_df.shape))
  tweets_only = tweets_df.iloc[0:,2:]

  # tweets preprocessing 
  tweets_only['Tweets'] = tweets_only['Tweets'].str.replace('http\S+|www.\S+', '', case=False)#stripping web addresses
  tweets_only['Tweets'] = tweets_only['Tweets'].str.replace('@','',)#striiping @
  tweets_only['Tweets'] = tweets_only['Tweets'].str.replace('\n','')#stripping \n
  #replacing amp&
  tweets_only['Tweets'] = tweets_only['Tweets'].str.replace("&amp", "and")
  
  #writing tweets to a text file 
  tweets_np = tweets_only['Tweets'].to_numpy()
  np.savetxt(f"{username}" +'_tweets.txt', tweets_np, fmt = "%s")

In [ ]:
pulltweets('sama')

Total number of tweets pulled 3196
Shape of tweeets dataframe (3196, 3)


###Joining the tweets text files

In [ ]:
tweet_files = ["simonsinek_tweets.txt","TomBilyeu_tweets.txt", "sama_tweets.txt", "naval_tweets.txt"]

In [ ]:
with open('/content//tweets_joined.txt', 'w') as outfile:
    for fname in tweet_files:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

In [ ]:
cols = ['Tweets']
df_tweets_joined = pd.read_csv('tweets_joined.txt',sep = "\n",error_bad_lines=False,names= cols)

In [ ]:
df_tweets_joined.shape

(12388, 1)

In [ ]:
df_tweets_joined

,Tweets
0,"If we were good at everything, we’d have no ne..."
1,"A small team, committed to a cause bigger than..."
2,"We are living in a polarized world. There is, ..."
3,Intelligence uses what is known to solve probl...
4,Great leaders don't see themselves as great; t...
...,...
12383,RT mattvcummings: There Are no “Adults”Everyon...
12384,EbrahimAsh EHSANI22 Astonishing coincidence. T...
12385,256_philosopher Quality over quantity. You wil...
12386,"RT NavalBot: ""A taste of freedom can make you ..."


In [ ]:
df_tweets_joined.to_csv('joined_tweets.csv')

##Using text search 


In [ ]:
#api call 
text_query = 'Flowrite'
count = -1
try:
 # Creation of query method using parameters
 tweets = tweepy.Cursor(api.search,q=text_query).items(count)
except BaseException as e:
    print('failed on_status,',str(e))
    time.sleep(3)

In [ ]:
 # Pulling information from tweets iterable object
 tweets_list_text = [[tweet.created_at, tweet.id, tweet.text] for tweet in tweets]
 
 # Creation of dataframe from tweets list
 # Add or remove columns as you remove tweet information
 tweets_df_text = pd.DataFrame(tweets_list_text)

In [ ]:
tweets_df_text

,0,1,2
0,2020-11-28 01:43:06,1332500230994948097,IYN さんでやってる夏目レモンさんの個展、結構年末ギリギリまでやってるんや。帰省したときに...
1,2020-11-27 22:28:50,1332451343592669185,今日は晴れてていい朝だ
2,2020-11-27 15:18:29,1332343040799694850,結局なにが言いたいかというとこれからも頑張って成長するのでどうぞよろしくお願いしますという。...
3,2020-11-27 15:14:29,1332342034309419014,今まではただこれが描きたいと思ったものをなんとか描き出すので精一杯だったけど、最近どんな線で...
4,2020-11-27 15:07:28,1332340269996380163,ただかわいらしい絵が描きたいわけではないんだよね。基本的にはどちらかというとかっこいい絵が描...
5,2020-11-27 15:02:03,1332338906721779713,絵を描き始めて3年半、今年は特にいろいろ実験してきた。今はちょうど次のステップに行く過渡期な...
6,2020-11-27 12:21:19,1332298455939653638,ホルベインの透明水彩絵具のサップグリーンとコバルトブルー ヒューとバーントアンバーとクリムソ...
7,2020-11-27 10:53:37,1332276386510213120,こういうのもおもしろいね\n\n#イラスト https://t.co/Dvt6uSldTU
8,2020-11-27 10:52:59,1332276228011618304,名刺の画像。最近梨ばかり上げている気がする\n\n#イラスト\n#イラスト好きさんと繋がりた...
9,2020-11-27 10:25:09,1332269222274088966,AAA GALLERYさまでの展示、今日まででした。見てくださった方々、ありがとうございます...
